# Goals, Actions, Information, Languages

* goals
    * persona
    * rules
    * process
* actions
    * available actions & tools
* information
    * task
    * history
    * inputs
    * environment state
* language

* G, A, L -> system messages, I -> user messages

## Tools

* tools & actions available to LLM, to interact with the environment
* tools (open) vs action (explicit, finite number of options)
* both need to be properly named & described/specified, to be understand and correctly leveraged by LLM
    * are tools optional or to be used in order, etc?
* feedback from tools/environment critical for further actions

## Tool Design and Naming Best Practices for AI Agents

### Tool Design

* provide a **limited, well-defined set of functions** specific to the agent’s task  
* reduce ambiguity and improve reliability  
* when starting, **prefer more specific tools**, flexibility can be added later  

#### Generic vs. Task-Specific

* generic
    * `list_files(directory: str)`
    * `read_file(file_path: str)`
    * `write_file(file_path: str, content: str)`
* task-specific
    * `list_python_files()` – returns Python files only from `src/`
    * `read_python_file(file_name: str)` – reads a Python file only from `src/`
    * `write_documentation(file_name: str, content: str)` – writes documentation only to `docs/`

Specific tools make purpose clear and reduce agent error.

---

### Tool Specification

* use **structured metadata**, for example JSON Schema, to define parameters and enforce correct usage  

* example: read a Python file
    ```json
    {
      "tool_name": "read_python_file",
      "description": "Reads the content of a Python file from the src/ directory.",
      "parameters": {
        "type": "object",
        "properties": {
          "file_name": { "type": "string" }
        },
        "required": ["file_name"]
      }
    }
    ```
* example: write documentation
    ```json
    {
      "tool_name": "write_documentation",
      "description": "Writes a documentation file to the docs/ directory.",
      "parameters": {
        "type": "object",
        "properties": {
          "file_name": { "type": "string" },
          "content": { "type": "string" }
        },
        "required": ["file_name", "content"]
      }
    }
    ```

---

### Naming

* names must clearly describe the action  
* avoid abbreviations and cryptic acronyms  

| poor name | better name |
|-----------|------------|
| `list_pf` | `list_python_files` |
| `rd_f`    | `read_python_file`  |
| `wrt_doc` | `write_documentation` |

---

### Error Handling

* tools should fail gracefully and return **structured, informative error messages**  
* example with Python file reading:
    ```python
    import os

    def read_python_file(file_name):
        """Reads a Python file from the src/ directory with error handling."""
        file_path = os.path.join("src", file_name)

        if not file_name.endswith(".py"):
            return {"error": "Invalid file type. Only Python files can be read."}

        if not os.path.exists(file_path):
            return {"error": f"File '{file_name}' does not exist in the src/ directory."}

        with open(file_path, "r") as f:
            return {"content": f.read()}
    ```

* ensure only `.py` files are read  
* return clear messages when file is missing  
* responses structured for easy parsing  

#### Adding instructions in error messages

* provide **just-in-time guidance** inside the error itself:
    ```python
    if not file_name.endswith(".py"):
        return {
          "error": "Invalid file type. Only Python files can be read. "
                   "Call list_python_files() to get valid file names."
        }
    ```
* the agent receives instructions exactly when needed without storing extra rules

---

### Key Principles

* use descriptive names  
* provide structured metadata (JSON Schema)  
* make tool intent obvious  
* implement robust error handling  
* return informative error messages  
* inject guidance directly in error messages when appropriate  

---